In [1]:
import xml.etree.ElementTree as ET
import os
import glob
import shutil
import random
import json
import yaml
import cv2
import easyocr
from tqdm import tqdm

### 보행자 데이터 전처리

In [25]:
city = ['서울특별시']
gu = ['영등포구', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구','은평구', \
  '서대문구', '마포구', '양천구', '강서구', '구로구','금천구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']
weather = ['맑음']
gan = ['주간', '야간']
sil = ['실내', '실외']

In [23]:
root_folder = '/home/pepsi/Downloads/pedestrian_detection/label'
change_folder = '/home/pepsi/Downloads/pedestrian_detection/labels'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = os.listdir(root_folder)
for file in file_list:
  name, ext = file.split('.')
  p = name.split("_")
  p[1] = str(city.index(p[1]))
  p[3] = f"{gu.index(p[3]):02d}"
  p[4] = str(weather.index(p[4]))
  p[5] = str(gan.index(p[5]))
  p[6] = str(sil.index(p[6]))
  new_name = "_".join(p) + '.' + ext
  shutil.copyfile(os.path.join(root_folder, file), os.path.join(change_folder, new_name))

In [27]:
root_folder = '/home/pepsi/Downloads/pedestrian_detection/image'
change_folder = '/home/pepsi/Downloads/pedestrian_detection/images'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = os.listdir(root_folder)
for file in file_list:
  name, ext = file.split('.')
  p = name.split("_")
  p[1] = str(city.index(p[1]))
  p[3] = f"{gu.index(p[3]):02d}"
  p[4] = str(weather.index(p[4]))
  p[5] = str(gan.index(p[5]))
  p[6] = str(sil.index(p[6]))
  new_name = "_".join(p) + '.' + ext
  shutil.copyfile(os.path.join(root_folder, file), os.path.join(change_folder, new_name))

In [6]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = float(((bbox[2] + bbox[0]) / 2) / w)
    y_center = float(((bbox[3] + bbox[1]) / 2) / h)
    width = float((bbox[2] - bbox[0]) / w)
    height = float((bbox[3] - bbox[1]) / h)
    return [x_center, y_center, width, height]

In [30]:
label_folder = '/home/pepsi/Downloads/pedestrian_detection/labels/'
new_folder = '/home/pepsi/Downloads/pedestrian_detection/new_labels/'

if not os.path.isdir(new_folder):
    os.mkdir(new_folder)

classes = ['pedestrian']
label_list = os.listdir(label_folder)
for file in label_list:
    with open(os.path.join(label_folder, file), 'r') as json_file:
        lab = json.load(json_file)
        h = int(lab['camera']['resolution_height'])
        w = int(lab['camera']['resolution_width'])
        f = open(os.path.join(new_folder, file[:-4]+'txt'),'a')
        for l in lab['annotations']:
            if l['label'] == '보행자':
                idx = 0
                points = l['points']
                yolo_bbox = xml_to_yolo_bbox([int(points[0][0]), int(points[0][1]), int(points[2][0]), int(points[2][1])], w,h)
                bbox_string = " ".join([str(x) for x in yolo_bbox])
                result = f"{idx} {bbox_string}\n"
                f.write(result)
        f.close()    

In [32]:
if os.path.isdir('/home/pepsi/Downloads/pedestrian_detection/train'):
    shutil.rmtree('/home/pepsi/Downloads/pedestrian_detection/train')
if os.path.isdir('/home/pepsi/Downloads/pedestrian_detection/val'):    
    shutil.rmtree('/home/pepsi/Downloads/pedestrian_detection/val')

folder_list = ['/home/pepsi/Downloads/pedestrian_detection/train', '/home/pepsi/Downloads/pedestrian_detection/val', '/home/pepsi/Downloads/pedestrian_detection/train/images', '/home/pepsi/Downloads/pedestrian_detection/train/labels', '/home/pepsi/Downloads/pedestrian_detection/val/images', '/home/pepsi/Downloads/pedestrian_detection/val/labels']
for folder in folder_list:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [33]:
random.seed(2020)

root_folder = "/home/pepsi/Downloads/pedestrian_detection"
file_folder = '/home/pepsi/Downloads/pedestrian_detection/new_labels'

file_list = os.listdir(file_folder)
random.shuffle(file_list)
test_ratio = 0.1

val_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

for i in val_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'val/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'val/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

for i in train_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'train/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'train/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

### 속도 표지판 전처리

In [49]:
road_sign_root = "/home/pepsi/Downloads/road_sign"
annot_path = os.path.join(road_sign_root,"annotations")
img_path = os.path.join(road_sign_root,"images")
label_path = os.path.join(road_sign_root,"labels")

if not os.path.exists(label_path):
    os.makedirs(label_path)

In [50]:
reader = easyocr.Reader(['en'], gpu=False)

speed_class_map = {
    30: 2,
    60: 3
}

files = glob.glob(os.path.join(annot_path, '*.xml'))
for fil in tqdm(files):
    filename = os.path.splitext(os.path.basename(fil))[0]
    img_file = os.path.join(img_path, f"{filename}.png")
    if not os.path.exists(img_file):
        continue

    img = cv2.imread(img_file)
    if img is None:
        continue

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    result = []

    for obj in root.findall('object'):
        if obj.find("name").text.lower() != 'speedlimit':
            continue

        bbox = [int(obj.find("bndbox").find(tag).text) for tag in ['xmin', 'ymin', 'xmax', 'ymax']]
        xmin, ymin, xmax, ymax = bbox
        cropped = img[ymin:ymax, xmin:xmax]

        # OCR 적용
        ocr_result = reader.readtext(cropped, detail=0)
        digits = [int(txt) for txt in ocr_result if txt.isdigit()]

        if not digits:
            continue

        speed = digits[0]
        if speed not in speed_class_map:
            continue

        class_index = speed_class_map[speed]
        yolo_bbox = xml_to_yolo_bbox(bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{class_index} {bbox_string}")

    if result:
        os.makedirs(label_path, exist_ok=True)
        with open(os.path.join(label_path, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

Using CPU. Note: This module is much faster with a GPU.


100%|██████████| 877/877 [01:30<00:00,  9.66it/s]


In [51]:
road_sign_path = "/home/pepsi/Downloads/road_sign"
label_ = '.txt'
img_ = '.png'

# 저장 경로 만들기
folder_list = ['/home/pepsi/Downloads/road_sign/road_sign_detection/train', '/home/pepsi/Downloads/road_sign/road_sign_detection/val', '/home/pepsi/Downloads/road_sign/road_sign_detection/train/images', \
                '/home/pepsi/Downloads/road_sign/road_sign_detection/train/labels', '/home/pepsi/Downloads/road_sign/road_sign_detection/val/images', '/home/pepsi/Downloads/road_sign/road_sign_detection/val/labels']
for folder in folder_list:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [52]:
file_list = os.listdir(os.path.join(road_sign_path, 'images'))
random.shuffle(file_list)

test_ratio = 0.1
test_list = file_list[:int(len(file_list) * test_ratio)]
train_list = file_list[int(len(file_list) * test_ratio):]

print(f"train의 개수 : {len(train_list)}, test의 개수 : {len(test_list)}")

# 경로 미리 설정
val_img_dir = '/home/pepsi/Downloads/road_sign/road_sign_detection/val/images'
val_label_dir = '/home/pepsi/Downloads/road_sign/road_sign_detection/val/labels'
train_img_dir = '/home/pepsi/Downloads/road_sign/road_sign_detection/train/images'
train_label_dir = '/home/pepsi/Downloads/road_sign/road_sign_detection/train/labels'

def copy_if_label_exists(file_list, dest_img_dir, dest_label_dir):
    for i in file_list:
        f_name = os.path.splitext(i)[0]
        label_file = os.path.join(road_sign_path, 'labels', f_name + label_)
        if not os.path.exists(label_file):
            # 라벨 파일 없으면 스킵
            continue

        src_img = os.path.join(road_sign_path, 'images', f_name + img_)
        dst_img = os.path.join(dest_img_dir, f_name + img_)
        dst_label = os.path.join(dest_label_dir, f_name + label_)

        shutil.copyfile(src_img, dst_img)
        shutil.copyfile(label_file, dst_label)

# 복사 수행
copy_if_label_exists(test_list, val_img_dir, val_label_dir)
copy_if_label_exists(train_list, train_img_dir, train_label_dir)

train의 개수 : 790, test의 개수 : 87


### 제작 데이터 전처리

In [3]:
root_folder = '/home/pepsi/Downloads/etc_detection/'
change_folder = '/home/pepsi/Downloads/etc_detection/labels'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = glob.glob(root_folder + "*.json")
file_list
for file in file_list:
  fiie = file.split('/')[-1]
  shutil.move(os.path.join(root_folder, file), os.path.join(change_folder, fiie))

In [4]:
root_folder = '/home/pepsi/Downloads/etc_detection/'
change_folder = '/home/pepsi/Downloads/etc_detection/images'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = glob.glob(root_folder + "*.png")
file_list
for file in file_list:
  fiie = file.split('/')[-1]
  shutil.move(os.path.join(root_folder, file), os.path.join(change_folder, fiie))

In [8]:
label_folder = '/home/pepsi/Downloads/etc_detection/labels/'
new_folder = '/home/pepsi/Downloads/etc_detection/new_labels/'

if not os.path.isdir(new_folder):
    os.mkdir(new_folder)

classes = ['pedestrian']
label_list = os.listdir(label_folder)
for file in label_list:
    with open(os.path.join(label_folder, file), 'r') as json_file:
        lab = json.load(json_file)
        h = int(lab['imageWidth'])
        w = int(lab['imageHeight'])
        f = open(os.path.join(new_folder, file[:-4]+'txt'),'a')
        for l in lab['shapes']:
            if l['label'] == '정지선':
                idx = 4
            elif l['label'] == '횡단보도':
                idx = 5
            elif l['label'] == '빨간불':
                idx = 6
            elif l['label'] == '노란불':
                idx = 7
            elif l['label'] == '초록불':
                dix = 8
            else:
                continue
            
            points = l['points']
            yolo_bbox = xml_to_yolo_bbox([int(points[0][0]), int(points[0][1]), int(points[2][0]), int(points[2][1])], w,h)
            bbox_string = " ".join([str(x) for x in yolo_bbox])
            result = f"{idx} {bbox_string}\n"
            f.write(result)
        f.close()    

In [9]:
if os.path.isdir('/home/pepsi/Downloads/etc_detection/train'):
    shutil.rmtree('/home/pepsi/Downloads/etc_detection/train')
if os.path.isdir('/home/pepsi/Downloads/etc_detection/val'):    
    shutil.rmtree('/home/pepsi/Downloads/etc_detection/val')

folder_list = ['/home/pepsi/Downloads/etc_detection/train', '/home/pepsi/Downloads/etc_detection/val', '/home/pepsi/Downloads/etc_detection/train/images', '/home/pepsi/Downloads/etc_detection/train/labels', '/home/pepsi/Downloads/etc_detection/val/images', '/home/pepsi/Downloads/etc_detection/val/labels']
for folder in folder_list:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [10]:
random.seed(2020)

root_folder = "/home/pepsi/Downloads/etc_detection"
file_folder = '/home/pepsi/Downloads/etc_detection/new_labels'

file_list = os.listdir(file_folder)
random.shuffle(file_list)
test_ratio = 0.1

val_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

for i in val_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'val/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'val/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

for i in train_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'train/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'train/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

In [61]:
data =dict()

data['train'] = '/home/pepsi/dev_ws/ros-repo-5/admin/src/ai_server/ai_train/dataset/drive_detection/train'
data['val'] = '/home/pepsi/dev_ws/ros-repo-5/admin/src/ai_server/ai_train/dataset/drive_detection/val'
data['test'] = ''

data['nc'] = 9
data['names'] =['Pedestrian','Car', '30Limit', '60Limit', 'Stop', 'Crosswlaker', 'redTrafficSign', 'yelloTrafficSign', 'greenTrafficSign']

with open('/home/pepsi/dev_ws/ros-repo-5/admin/src/ai_server/ai_train/dataset/drive_detection/car_detection.yaml', 'w') as f:
	yaml.dump(data, f)